# Find out on which weekday people bike the most with groupby and aggregate

In [ ]:
SYFT_VERSION = ">=0.9,<1.0.0"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -q

In [ ]:
# syft absolute
import syft as sy

sy.requires(SYFT_VERSION)

In [ ]:
server = sy.orchestra.launch(name="pandas-test-datasite-4", port=9084, reset=True)

# Data owner: upload data

In [ ]:
root_datasite_client = server.login(email="info@openmined.org", password="changethis")

## Load data

In [ ]:
# stdlib
from datetime import timedelta

# third party
from dateutil.parser import parse
import pandas as pd
from pandas._libs.tslibs.timestamps import Timestamp

# syft absolute
from syft.service.project.project import Project
from syft.util.util import PANDAS_DATA
from syft.util.util import autocache

In [ ]:
df = pd.read_csv(
    autocache(f"{PANDAS_DATA}/bikes.csv"),
    sep=";",
    encoding="latin1",
    parse_dates=["Date"],
    dayfirst=True,
    index_col="Date",
)
df[:3]

In [ ]:
assert len(df) == 310

## Create mock data

In [ ]:
# stdlib
from random import randint

# third party
import numpy as np


def get_mock_int(i, var=10):
    base = 100 + int(np.cos(i / 10) * 50)
    return randint(max(base - var, 1), max(1, base + var))


def get_mock_date(i):
    return str(parse("Jun 1 2005").date() + timedelta(days=i))


# some randomness
r = [(randint(1, 40), randint(1, 40)) for _ in range(7)]


def get_mock_row(i):
    res = [get_mock_int(i + r[j][0], r[j][1]) for j in range(7)]
    # add NaNs in the second and 10th col
    res.insert(1, None), res.insert(9, None)
    return res

In [ ]:
mock = pd.DataFrame(
    index=[Timestamp(get_mock_date(i)) for i in range(len(df))],
    data=[get_mock_row(i) for i in range(len(df))],
    columns=df.columns,
)

Upload the data

In [ ]:
dataset = sy.Dataset(
    name="bikes2",
    asset_list=[sy.Asset(name="bikes", data=df, mock=mock, mock_is_real=False)],
)
root_datasite_client.upload_dataset(dataset)

## create Data scientist

In [ ]:
user = root_datasite_client.register(
    name="Jane Doe",
    email="jane@caltech.edu",
    password="abc123",
    password_verify="abc123",
    institution="Caltech",
    website="https://www.caltech.edu/",
)

# todo: give user data scientist role

guest_datasite_client = server.client

guest_client = guest_datasite_client.login(email="jane@caltech.edu", password="abc123")

## Create user account

# Data Scientist: create syft_function

## Download mock and submit project

### Get mock

In [ ]:
ds = guest_datasite_client.datasets[0]

In [ ]:
asset = ds.assets["bikes"]

In [ ]:
asset

In [ ]:
bikes = asset.mock

### Adding a 'weekday' column to our dataframe

First, we need to load up the data. We've done this before.

In [ ]:
bikes["Berri 1"].plot()

Next up, we're just going to look at the Berri bike path. Berri is a street in Montreal, with a pretty important bike path. I use it mostly on my way to the library now, but I used to take it to work sometimes when I worked in Old Montreal.

So we're going to create a dataframe with just the Berri bikepath in it

In [ ]:
berri_bikes = bikes[["Berri 1"]].copy()

In [ ]:
berri_bikes[:5]

Next, we need to add a 'weekday' column. Firstly, we can get the weekday from the index. We haven't talked about indexes yet, but the index is what's on the left on the above dataframe, under 'Date'. It's basically all the days of the year.

In [ ]:
berri_bikes.index

You can see that actually some of the days are missing -- only 310 days of the year are actually there. Who knows why.

Pandas has a bunch of really great time series functionality, so if we wanted to get the day of the month for each row, we could do it like this:

In [ ]:
berri_bikes.index.day

We actually want the weekday, though:

In [ ]:
berri_bikes.index.weekday

These are the days of the week, where 0 is Monday. I found out that 0 was Monday by checking on a calendar.

Now that we know how to get the weekday, we can add it as a column in our dataframe like this:

In [ ]:
berri_bikes.loc[:, "weekday"] = berri_bikes.index.weekday
berri_bikes[:5]

### Adding up the cyclists by weekday

This turns out to be really easy!

Dataframes have a `.groupby()` method that is similar to SQL groupby, if you're familiar with that. I'm not going to explain more about it right now -- if you want to to know more, the [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) is really good.

In this case, `berri_bikes.groupby('weekday').aggregate(sum)` means "Group the rows by weekday and then add up all the values with the same weekday".

In [ ]:
weekday_counts = berri_bikes.groupby("weekday").aggregate(sum)
weekday_counts

It's hard to remember what 0, 1, 2, 3, 4, 5, 6 mean, so we can fix it up and graph it:

In [ ]:
# weekday_counts.index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
# weekday_counts

In [ ]:
weekday_counts.plot(kind="bar")

So it looks like Montrealers are commuter cyclists -- they bike much more during the week. Neat!

### Putting it together

Now we want to request the full code execution.

Let's put all that together, to prove how easy it is. 6 lines of magical pandas!

If you want to play around, try changing sum to max, numpy.median, or any other function you like.

In [ ]:
# Todo, fix indexes in function

In [ ]:
@sy.syft_function(
    input_policy=sy.ExactMatch(bikes=asset.pointer),
    output_policy=sy.SingleExecutionExactOutput(),
)
def get_bike_hist(bikes):
    berri_bikes = bikes[["Berri 1"]].copy()
    berri_bikes.loc[:, "weekday"] = berri_bikes.index.weekday
    weekday_counts = berri_bikes.groupby("weekday").aggregate(sum)
    return weekday_counts

Create and submit project

In [ ]:
new_project = sy.Project(
    name="Pandas Chapter 4",
    description="Hi, I would like to plot histogram of bikers per weekday",
    members=[guest_client],
)
new_project

In [ ]:
project = new_project.send()
assert isinstance(project, sy.service.project.project.Project)
project

In [ ]:
project.create_code_request(get_bike_hist, guest_client)

In [ ]:
assert len(guest_client.code.get_all()) == 1

In [ ]:
assert len(project.events) == 1

In [ ]:
assert isinstance(project.events[0], sy.service.project.project.ProjectRequest)

# Data owner: approve request

In [ ]:
# syft absolute

In [ ]:
root_datasite_client = server.login(email="info@openmined.org", password="changethis")

## Get notifications

In [ ]:
notifications = root_datasite_client.notifications.get_all_unread()

In [ ]:
notifications

In [ ]:
project_notification = [
    x for x in notifications if issubclass(x.linked_obj.object_type, Project)
][0]

In [ ]:
request = project_notification.link.events[0].request
func = request.code

In [ ]:
func

In [ ]:
get_col_user_function = func.run

In [ ]:
real_data = root_datasite_client.datasets[0].assets[0].data

In [ ]:
real_result = get_col_user_function(bikes=real_data)

In [ ]:
real_result[:3]

In [ ]:
result = request.approve()
assert isinstance(result, sy.SyftSuccess)

# Data scientist: compute result

In [ ]:
asset = guest_client.datasets[0].assets[0]

In [ ]:
guest_client.code[0].status

In [ ]:
result_ptr = guest_client.code.get_bike_hist(bikes=asset)
real_result = result_ptr.get()
real_result.plot(kind="bar")

In [ ]:
server.land()